In [3]:
# search for entities
from glob import glob 
from traceback import format_exc

url_fpaths = "datasets/*txt"

for url_fpath in glob(url_fpaths):
    print(url_fpath)
    with open(url_fpath, "r") as in_f:
        for url in in_f:
            try:
                url = db2wiki(url.strip())
                query = 'origins:"{}"'.format(url)
                r = make_query(query)
                s = json.loads(r.content)
                print(url, len(r.content))
            except KeyboardInterrupt:
                break
            except:
                print(url, "error")
                # print(format_exc())
                # retry here


datasets/dbpedia-url.txt
en.wikipedia.org/wiki/Anxiety 8541
en.wikipedia.org/wiki/Conspiracy_theory 10494
en.wikipedia.org/wiki/Consumer 13972
en.wikipedia.org/wiki/Internet 31287
en.wikipedia.org/wiki/Internet_privacy 2129
en.wikipedia.org/wiki/Lawsuit 3758
datasets/n3-reuters-128-urls.txt
en.wikipedia.org/wiki/Brazil 232363
en.wikipedia.org/wiki/Nippon_Telegraph_and_Telephone 6369
en.wikipedia.org/wiki/Ally_Financial 10644
en.wikipedia.org/wiki/Saddam_Hussein 15420
aksw.org/notInWiki/Motorola_Inc 63
en.wikipedia.org/wiki/Japan 309706
datasets/kore50-urls.txt
en.wikipedia.org/wiki/David_Beckham 20740
en.wikipedia.org/wiki/Victoria_Beckham 17703
en.wikipedia.org/wiki/David_Beckham 20739


In [15]:
d = s["data"]
d[]

{'allNames': ['دیوید بکهام',
  'დევიდ ბექჰემი',
  'डेविड बेखम',
  'Deivids Bekhems',
  'ډېوېډ بکهم',
  'Дэвид Бекҳэм',
  '大衛·碧咸',
  '大卫·贝克汉姆',
  'David Beckham OBE',
  'Devid Bekhem',
  '貝克漢',
  'דייוויד בקהאם',
  'デビッド・ベッカム',
  'Дэйвід Бэкгэм',
  '大衛碧咸',
  'Posh & Becks',
  'دێڤید بێکام',
  'ଡେଭିଡ ବେକହାମ',
  'Ντέιβιντ Μπέκαμ',
  '大衛·貝克漢',
  'Дејвид Бекам',
  'Becks',
  'เดวิด เบคแคม',
  'Дэвід Бекхэм',
  '데이비드 베컴',
  'ಡೇವಿಡ್ ಬೆಕ್\u200cಹ್ಯಾಮ್',
  'David J. Beckham',
  'ডেভিড বেকহ্যাম',
  'Дейвид Бекъм',
  'ଡେଭିଡ ରବର୍ଟ ଜୋସେଫ ବେକହାମ',
  'Դևիդ Բեքհեմ',
  "Posh 'n' Becks",
  'டேவிட் பெக்காம்',
  'ديفيد بيكهام',
  'David Robert Joseph Beckham',
  'Девід Бекхем',
  'డేవిడ్ బెక్హాం',
  'Дэвид Бекхэм',
  'ਡੇਵਿਡ ਬੈਕਮ',
  'ဒေးဗစ်ဘက်ခ်ဟမ်',
  'डेव्हिड बेकहॅम',
  'דעיוויד בעקהעם',
  'ડેવિડ બેકહામ',
  'डेभिड बेक्ह्याम',
  'ديفد بيكام',
  '碧咸',
  'ഡേവിഡ് ബെക്കാം',
  'David Beckham'],
 'allUris': ['plus.google.com/+davidbeckham',
  'facebook.com/beckham',
  'davidbeckham.com',
  'youtube.com/user/dav

# DQL

In [2]:
import requests
from pprint import pprint
import json 
import codecs


endpoint_diffbot = "http://kg.diffbot.com/kg/dql_endpoint"


def db2wiki(url, to_en=True):
    
    url = url.replace("https://", "")
    url = url.replace("http://", "")
    
    if to_en:
        wiki_domain = "en.wikipedia.org/wiki/"
    else:
        wiki_domain = "wikipedia.org/wiki/"
        
    new_url = url.replace("dbpedia.org/resource/", wiki_domain)
    if new_url == url:
        new_url = url.replace("dbpedia.org/page/", wiki_domain)
        
        
    return new_url

def get_token():
    with open("../.diffbot-token", "r") as f:
        return f.read().strip()


def make_query(query):
    data = {
        "token": get_token(),
        "query": query,
        "type": "query"}
    r = requests.get(endpoint_diffbot, params=data)

    return r 


def save2json(output_fpath, r):
    with codecs.open(output_fpath, "w", "utf-8") as out:   
        out.write(json.dumps(json.loads(r.content)))
    print(output_fpath)
 

def query_and_save(query, output_fpath):
    r = make_query(query)
    save2json(output_fpath, r)
    
    
entity_types = ["AdministrativeArea", "Article", "Corporation",
                    "DegreeEntity", "EducationMajorEntity", "EducationalInstitution",
                   "EmploymentCategory", "Image", "Intangible", "Landmark", "LocalBusiness",
                   "Miscellaneous", "Organization", "Person", "Place", "Product", "Role",
                   "Skill", "Video"]

In [ ]:
query_and_save('allUris:"barackobama.com"', "data/all-uris.json")
query_and_save('wikipediaUri:"en.wikipedia.org/wiki/Barack_Obama"', "data/wiki-uri.json")
query_and_save('allUris:"en.wikipedia.org/wiki/Barack\_Obama"', "data/all-uris-wiki.json")
query_and_save('origins:"en.wikipedia.org/wiki/Barack_Obama"', "data/origins.json")

In [ ]:
for entity_type in entity_types:
    query_and_save(
        query='type:{}'.format(entity_type),
        output_fpath="data/{}.json".format(entity_type))
    
query_and_save(
    query='type:Person name:"Alexander Panchenko"',
    output_fpath="data/ap.json")


query_and_save(
    query='type:Person employments.employer.name:"Diffbot"',
    output_fpath="data/diffbot-employees.json")


query_and_save(
    query='type:Person employments.{title:"CEO" employer.name:"Diffbot"}',
    output_fpath="data/diffbot-ceo.json")

query_and_save(
    query='type:Person employments.{employer.name:"Diffbot" isCurrent:true}',
    output_fpath="data/diffbot-current-employees.json")


# Testing type of links

In [ ]:
query_and_save(
    query='type:Person name:"Angela Merkel"',
    output_fpath="data/am.json")

query_and_save(
    query='type:Person name:"Barack Obama"',
    output_fpath="data/bo.json")

query_and_save(
    query='type:Person name:"Nicolas Sarkozy"',
    output_fpath="data/ns.json")

query_and_save(
    query='type:Person name:"Diego Maradona"',
    output_fpath="data/dm.json")